#提取网页的内链与外链

In [1]:
from urllib.request import  urlopen
from bs4 import BeautifulSoup
import re
import datetime
import random

In [2]:
pages=set()
random.seed(datetime.datetime.now())

In [26]:
def getInternalLinks(bs0bj,includeUrl):
    internalLinks=[]
    for link in bs0bj.findAll("a",href=re.compile("^(/|.*"+includeUrl+")$")):
        if link.attrs["href"] is not None:
            if link.attrs["href"] not in internalLinks:
                internalLinks.append(link.attrs["href"])
    return internalLinks

In [42]:
def getExternalLinks(bs0bj,excludeUrl):
    externalLinks=[]
    for link in bs0bj.findAll("a",href=re.compile("^(http|www)((?!"+excludeUrl+").)*$")):
        if link.attrs["href"] is not None:
            if link.attrs["href"] not in externalLinks:
                externalLinks.append(link.attrs["href"])
    return externalLinks

In [43]:
def splitAddress(address):
    addressParts=address.replace("http://","").split("/")
    return addressParts

In [44]:
def getRandomExternalLink(startingPage):
    html=urlopen(startingPage)
    bs0bj=BeautifulSoup(html)
    externalLinks=getExternalLinks(bs0bj,splitAddress(startingPage)[0])
    if len(externalLinks) == 0:
        internalLinks=getInternalLinks(startingPage)
        return getNextExternalLink(internalLinks[random.randint(0,len(internalLinks)-1)])
    else:
        return externalLinks[random.randint(0,len(externalLinks)-1)]

In [50]:
def followExternalOnly(startingSite):
    externalLink=getRandomExternalLink("http://oreilly.com")
    print("随机外链："+externalLink)
    followExternalOnly(externalLink)

#产生一个随机外链

In [51]:
followExternalOnly("http://oreilly.com")

C:\Users\liuzhiwei\Anaconda2\envs\py3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\liuzhiwei\Anaconda2\envs\py3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


随机外链：http://twitter.com/oreillymedia
随机外链：https://www.youtube.com/user/OreillyMedia
随机外链：http://twitter.com/oreillymedia
随机外链：https://cdn.oreillystatic.com/pdf/oreilly_high_performance_organizations_whitepaper.pdf
随机外链：http://fb.co/OReilly
随机外链：https://www.linkedin.com/company/oreilly-media
随机外链：https://www.linkedin.com/company/oreilly-media
随机外链：https://cdn.oreillystatic.com/pdf/oreilly_high_performance_organizations_whitepaper.pdf
随机外链：https://cdn.oreillystatic.com/pdf/oreilly_high_performance_organizations_whitepaper.pdf


ConnectionAbortedError: [WinError 10053] 你的主机中的软件中止了一个已建立的连接。

#可能遇到的异常情况：
1.需要设置代理访问
2.网页验证问题

#收集所有内外链

In [63]:
allExtLinks=set()
allIntLinks=set()

In [65]:
def getAllLinks(siteUrl):
    html= urlopen(siteUrl)
    bs0bj=BeautifulSoup(html)
    internalLinks=getInternalLinks(bs0bj,splitAddress(siteUrl)[0])
    externalLinks=getExternalLinks(bs0bj,splitAddress(siteUrl)[0])
    for link in internalLinks:
        if link not in allIntLinks:
            allIntLinks.add(link)
            print("内链："+link)
            getAllLinks(link)
    for link in externalLinks:
        if link not in allExtLinks:
            allExtLinks.add(link)
            print("外链："+link)
            getAllLinks(link)
getAllLinks("http://oreilly.com")

C:\Users\liuzhiwei\Anaconda2\envs\py3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\liuzhiwei\Anaconda2\envs\py3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


内链：http://members.oreilly.com
外链：https://www.safaribooksonline.com/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170601+nav
外链：https://www.oreilly.com/
外链：http://www.oreilly.com/conferences/
外链：http://shop.oreilly.com/
内链：http://shop.oreilly.com
外链：https://members.oreilly.com/account/emedia
外链：http://members.oreilly.com
外链：http://www.oreilly.com/about/
内链：http://www.oreilly.com
外链：https://www.safaribooksonline.com/?utm_medium=content&utm_source=oreilly.com&utm_campaign=lgen&utm_content=20170505+homepage+get+started+now
外链：https://www.safaribooksonline.com/public/self-assessments/
外链：https://www.safaribooksonline.com/learning-paths/learning-path-git/9781491987254/8b4488de-43a5-4ffa-bf82-af1e19ee1b64
外链：https://twitter.com/safari


URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

#异常问题：
1.网络问题
2.无效地址